# Trabalho de Algoritmo Genético 
---------------------------------

**Tiago foi  ao  supermercado  com  R$ 80,00**,  com  o  objetivo  de  comprar  **20  itens** da  sua lista  de  compras,  após  sentir  uma  mistura  de  tristeza,  angústia  e  revolta,  devido  ao  preço excessivo  dos  produtos  que  ele  gostaria  de  comprar,  decidiu  analisar  o  quanto  ele  realmente necessitava  de  cada  um  dos  itens,  pois  infelizmente  não  conseguiria  comprar  todos  com  o dinheiro que possuía. Assim, Tiago atribuiu uma nota de **1 a 10 para cada item, de acordo com sua  necessidade**,  onde  a  nota  1  representa  pouca  necessidade  daquele  item  e  a  nota  10 representa que o item é muito necessário.

Após essa análise, foi possível gerar a tabela a seguir, onde cada item aparece associado  ao seu preço e a necessidade que Tiago  tem de comprá-lo.

|  Itens |          Produtos         |   Preço (R$) |  Necessidade  |
|:------:|:-------------------------:|:------------:|:-------------:|
|    1   | Arroz (5kg)               |     22       |      10       | 
|    2   | Açúcar (2kg)              |      3       |       8       |
|    3   | Óleo (1l)                 |      8       |       6       |
|    4   | Feijão (1kg)              |      5       |      10       |
|    5   | Macarrão (500g)           |      5       |       8       | 
|    6   | Sardinha (1lata)          |      5       |       7       | 
|    7   | Carne (1kg)               |     32       |       9       |
|    8   | Frango (1kg)              |     13       |       9       |
|    9   | Queijo (200g)             |      8       |       5       |
|   10   | Presunto (200g)           |      4       |       5       | 
|   11   | Pão (8un.)                |      6       |       6       |
|   12   | Banana (1kg)              |      4       |       7       | 
|   13   | Laranja (1kg)             |      2       |       7       | 
|   14   | Abacate (1kg)             |      7       |       2       |
|   15   | Sabão (1un.)              |      2       |       9       |
|   16   | Limpador multiuso (1un.)  |      4       |       6       | 
|   17   | Água Tônica (500ml)       |      7       |       1       |
|   18   | Polpa de Fruta (500ml)    |      6       |       4       | 
|   19   | Refrigerante (2l)         |      8       |       3       |
|   20   | Cerveja (600ml)           |      6       |       2       | 

Para  ajudar  Tiago  nesse  momento  difícil,  **implemente  um  algoritmo  genético  para encontrar a melhor combinação de itens, de modo a maximizar a soma da importância dos itens escolhidos, sem ultrapassar o valor disponível de R$ 80,00**


## Resolução Proposta

Para o problema proposto foi criada a `class Produtos`

|  Itens |          Produtos         |   Preço (R$) |  Necessidade  |
|:------:|:-------------------------:|:------------:|:-------------:|
| indice |           nome            |     valor    |      peso     |
|    1   |         Arroz (5kg)       |     22       |      10       | 

                Produto(" Arroz (5kg)", 10, 22) -->  Produto(nome, peso, valor):

In [28]:
class Produto():
    def __init__(self, nome, peso, valor):
        self.nome = nome
        self.peso = peso
        self.valor = valor

## Algoritmo Genético

o cromossomo é uma estrutura de dados que representa uma possível solução para o problema, cada bit da série (gene) representa alguma característica da solução (produto).

![ag](ag.png)

Cada indivíduo representa uma solução do problema (cromossomo)

In [25]:
class Individuo():
    def __init__(self, valores, pesos, limite_valores, geracao=0):
        self.valores = valores
        self.pesos = pesos
        self.limite_valores = limite_valores
        self.nota_fitness = 0
        self.valor_usado = 0
        self.geracao = geracao
        self.cromossomo = []
        
        for i in range(len(valores)):
            if random() < 0.5:
                self.cromossomo.append("0")
            else:
                self.cromossomo.append("1")


Para cada indivíduo é dada uma nota, que reflete sua habilidade de adaptação a determinado ambiente.

A função fitnesse fornece uma medida da proximidade da solução em relação as necessidades(pesos) e limite de valor, possibilitando calcular sua probabilidade de ser selecionado. 

In [ ]:

    def fitness(self):
        nota = 0
        soma_valores = 0
        for i in range(len(self.cromossomo)):
           if self.cromossomo[i] == '1':
               nota += self.pesos[i]
               soma_valores += self.valores[i]
        if soma_valores > self.limite_valores:
            nota = 1
        self.nota_fitness = nota
        self.valor_usado = soma_valores      


É feito o corte de cruzamento em ponto(s) determinado(s) pelo algoritmo, criando indivíduos com a combinação dos genes dos pais.

Esta recombinação garante que os melhores indivíduos sejam capazes de trocar entre si as informações que os levam a ser mais aptos a sobreviver, e assim gerar descendentes ainda mais aptos.

In [ ]:

    def crossover(self, outro_individuo):
            corte = round(random()  * len(self.cromossomo))

            filho1 = outro_individuo.cromossomo[0:corte] + self.cromossomo[corte::]
            filho2 = self.cromossomo[0:corte] + outro_individuo.cromossomo[corte::]

            filhos = [Individuo(self.valores, self.pesos, self.limite_valores, self.geracao + 1),
                      Individuo(self.valores, self.pesos, self.limite_valores, self.geracao + 1)]
            filhos[0].cromossomo = filho1
            filhos[1].cromossomo = filho2
            return filhos  


As mutações alteram sem relacionar com outro indivíduo, mudando aleatoriamente a descendência criada pelo cruzamento. Tendo como objetivo permitir maior variabilidade genética na população, impedindo que a busca fique estagnada em um mínimo local.

A taxa de mutação deve ser o suficiente para assegurar a diversidade de cromossomos na população

In [ ]:

    def mutacao(self, taxa_mutacao):
        for i in range(len(self.cromossomo)):
            if random() < taxa_mutacao:
                if self.cromossomo[i] == '1':
                    self.cromossomo[i] = '0'
                else:
                    self.cromossomo[i] = '1'
        return self

Uma população é o conjunto dos indivíduos.

In [26]:
class AlgoritmoGenetico():
    def __init__(self, tamanho_populacao):
        self.tamanho_populacao = tamanho_populacao
        self.populacao = []
        self.geracao = 0
        self.melhor_solucao = 0
        self.lista_solucoes = []


Uma população de n individuos é gerada aleatoriamente. 

In [ ]:

    def __inicializa_populacao__(self, valores, pesos, limite_valores):
        for i in range(self.tamanho_populacao):
            self.populacao.append(Individuo(valores, pesos, limite_valores))
        self.melhor_solucao = self.populacao[0]        

    def __ordena_populacao__(self):
        self.populacao = sorted(self.populacao,
                                key = lambda populacao: populacao.nota_fitness,
                                reverse = True)


O melhor indivíduo é aquele que obdece a todos os critérios de avaliação, e que possui a maior nota possível.

In [ ]:

    def __melhor_individuo__(self, individuo):
        if individuo.nota_fitness > self.melhor_solucao.nota_fitness:
            self.melhor_solucao = individuo

    def __soma_avaliacoes__(self):
        soma = 0
        for individuo in self.populacao:
           soma += individuo.nota_fitness
        return soma


algoritmo de **seleção por "roleta"**, onde os indivíduos são ordenados de acordo com a função de avaliação (fitness).

Dessa forma conseguimos privilegiar os mais bem adaptados (maior necessidade), sem deixar de lado a diversidade dos menos adaptados (roleta viciada)

In [ ]:

    def __selecao__(self, soma_fitness):
        pai = -1
        valor_sorteado = random() * soma_fitness
        soma = 0
        i = 0
        while i < len(self.populacao) and soma < valor_sorteado:
            soma += self.populacao[i].nota_fitness
            pai += 1
            i += 1
        return pai

    def __visualiza_geracao__(self):
        melhor = self.populacao[0]
        print("Geração:%s -> Avaliação: %s Valor: %s Cromossomo: %s" % (self.populacao[0].geracao,
                                                               melhor.nota_fitness,
                                                               melhor.valor_usado,
                                                               melhor.cromossomo))


Solucionando o problema com o uso do algoritmo genético

In [ ]:

    def __solucao__(self, taxa_mutacao, numero_geracoes, valores, pesos, limite_valores):
        self.__inicializa_populacao__(valores, pesos, limite_valores)
        
        for individuo in self.populacao:
            individuo.__fitness__()
        
        self.__ordena_populacao__()
        self.melhor_solucao = self.populacao[0]
        self.lista_solucoes.append(self.melhor_solucao.nota_fitness)
        
        self.__visualiza_geracao__()
        
        for geracao in range(numero_geracoes):
            soma_fitness = self.__soma_avaliacoes__()
            nova_populacao = []
            
            for individuos_gerados in range(0, self.tamanho_populacao, 2):
                genitor_1 = self.__selecao__(soma_fitness)
                genitor_2 = self.__selecao__(soma_fitness)
                
                filhos = self.populacao[genitor_1].__crossover__(self.populacao[genitor_2])
                
                nova_populacao.append(filhos[0].__mutacao__(taxa_mutacao))
                nova_populacao.append(filhos[1].__mutacao__(taxa_mutacao))
            
            self.populacao = list(nova_populacao)
            
            for individuo in self.populacao:
                individuo.__fitness__()
            
            self.__ordena_populacao__()
            
            self.__visualiza_geracao__()
            
            melhor = self.populacao[0]
            self.lista_solucoes.append(melhor.nota_fitness)
            self.__melhor_individuo__(melhor)

        print("-------------------------------------------------------")
        print("\nMelhor solução -> Geração: %s - Avaliação: %s  - Valor: %s \nCromossomo: %s" %
              (self.melhor_solucao.geracao,
               self.melhor_solucao.nota_fitness,
               self.melhor_solucao.valor_usado,
               self.melhor_solucao.cromossomo))
        print("-------------------------------------------------------")
        pago =  self.melhor_solucao.valor_usado
        troco = limite_valores -pago

        print("\nValor pago: R$ %s \nTroco: R$ %s \n" % ( pago, troco ))

        return self.melhor_solucao.cromossomo

## Dados:
Thiago foi  ao  supermercado  com  R$ 80,00
Sua lista  de  compras possui **20  itens**
Thiago atribuiu nota de **1 a 10 para cada item, de acordo com sua  necessidade**

onde  a  nota  1  representa  pouca  necessidade  daquele  item  e  a  nota  10 representa que o item é muito necessário.

In [33]:
import csv

from algoritmo_genetico import AlgoritmoGenetico
from produto import Produto


def __main__():

    lista_produtos = []

    with open("produtos.csv", encoding="utf-8") as file:
        lista = csv.reader(file, delimiter=",")
        for produto in lista:
            lista_produtos.append(
                Produto(produto[0], (int(produto[1])), (int(produto[2])))
            )

    nomes = []
    valores = []
    pesos = []

    for produto in lista_produtos:
        nomes.append(produto.nome)
        valores.append(produto.valor)
        pesos.append(produto.peso)

    limite = 80
    tamanho_populacao = len(lista_produtos)
    taxa_mutacao = 0.01
    numero_geracoes = 10 * tamanho_populacao

    algoritmo_genetico = AlgoritmoGenetico(tamanho_populacao)
    resultado = algoritmo_genetico.__solucao__(
        taxa_mutacao, numero_geracoes, valores, pesos, limite
    )

    for i, item in enumerate(lista_produtos):
        if resultado[i] == "1":
            print(
                "Produto: %s R$ %s " % (
                    item.nome, item.valor)
            )
            
    print("--------------------------------")
    print("Thiago ficou sem")
    for i, item in enumerate(lista_produtos):
        if resultado[i] == "0":
            print(
                "Produto: %s R$ %s " % (
                    item.nome, item.valor)
            )


if __name__ == "__main__":
    __main__()


Geração:0 -> Peso: 73 - Valor: 75  - Cromossomo: ['0', '1', '1', '0', '1', '1', '0', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1']
Geração:1 -> Peso: 73 - Valor: 72  - Cromossomo: ['0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0']
Geração:2 -> Peso: 75 - Valor: 78  - Cromossomo: ['0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '1']
Geração:3 -> Peso: 73 - Valor: 72  - Cromossomo: ['0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0']
Geração:4 -> Peso: 65 - Valor: 75  - Cromossomo: ['1', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '0']
Geração:5 -> Peso: 73 - Valor: 72  - Cromossomo: ['0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '1', '0', '1', '0']
Geração:6 -> Peso: 68 - Valor: 64  - Cromossomo: ['0', '0', '1', '0', '1', '1', '0', '1', '0', '1', 